In [1]:
from BikeStationCDF import BikeStationCDF as BSCDF
from BikeStations import BikeStationNetwork as BSN
from BikeRelocationScheme import BikeRelocationScheme as BRS

from RouteAnalysis import RouteAnalysis

import traci
from sumolib import checkBinary

from Geometry import GeometryClass

import matplotlib.pyplot as plt
import numpy as np

In [2]:
traci.init(58890)
traci.setOrder(2)

In [3]:
myBikeNetwork = BSN()

In [4]:
RouteAnalysisSUMO = RouteAnalysis()

In [5]:
AllStationsIds = myBikeNetwork.getAllStationOnNetwork()
RelocationSchemes = BRS(AllStationsIds).getRebalancingWeights()

In [6]:
# Station = AllStationsIds[3]
# WkDay = 0
# RelocationSchemes[Station][WkDay][BRS.C_WEIGHTINDEX]

In [7]:
# myBikeNetwork.BikeStationsDict.keys()

In [29]:
myBikeNetwork.getAllStationOnNetwork()

[2,
 9,
 80,
 81,
 77,
 75,
 78,
 74,
 76,
 58,
 61,
 59,
 60,
 57,
 62,
 53,
 50,
 51,
 54,
 56,
 55,
 52,
 38,
 36,
 37,
 34,
 35,
 39,
 26,
 25,
 31,
 29,
 27,
 28,
 8,
 6,
 12,
 10,
 11]

In [9]:
dayItinerary = myBikeNetwork.getDayItinerary(0)
# for idx, _ in enumerate(dayItinerary):
#     print('Itinerary for Station: ' + str(dayItinerary[idx][0][2]))

In [12]:
dayItinerary

[array([[  6.25 ,   1.   ,   2.   ,  -1.   ],
        [  6.375,   0.   ,   2.   ,  43.   ],
        [  6.75 ,   1.   ,   2.   ,  -1.   ],
        [  6.75 ,   1.   ,   2.   ,  -1.   ],
        [  6.875,   1.   ,   2.   ,  -1.   ],
        [  7.25 ,   1.   ,   2.   ,  -1.   ],
        [  7.5  ,   0.   ,   2.   ,  58.   ],
        [  7.625,   0.   ,   2.   ,  35.   ],
        [  7.625,   1.   ,   2.   ,  -1.   ],
        [  7.625,   1.   ,   2.   ,  -1.   ],
        [  7.75 ,   1.   ,   2.   ,  -1.   ],
        [  7.75 ,   0.   ,   2.   ,  56.   ],
        [  8.   ,   1.   ,   2.   ,  -1.   ],
        [  8.125,   0.   ,   2.   , 246.   ],
        [  8.125,   1.   ,   2.   ,  -1.   ],
        [  8.125,   0.   ,   2.   , 175.   ],
        [  8.125,   0.   ,   2.   ,  69.   ],
        [  8.25 ,   0.   ,   2.   ,  80.   ],
        [  8.25 ,   1.   ,   2.   ,  -1.   ],
        [  8.25 ,   0.   ,   2.   , 165.   ],
        [  8.375,   0.   ,   2.   ,  85.   ],
        [  8.375,   1.   ,   2.   

In [13]:
BikeInitialStatus = [s.availableBikes() for s in list(myBikeNetwork.BikeStationsDict.values())]

In [14]:
def updateStatusOfStations(time, dayItinerary):
    for idx in range(len(dayItinerary)):
        for eventsStation in range(len(dayItinerary[idx])):
            # StationId retrived from trip timestamps of dayItinerary
            StationId = int(dayItinerary[idx][eventsStation][2])
            # Process events only when they're equal to time
            # times are represented in seconds, they need to be multiplied
            # by C_MINUTES_IN_HOUR
            if(int(dayItinerary[idx][eventsStation][0] * C_MINUTES_IN_HOUR) == time):
                # departure or arrival trip type
                tripInstance = dayItinerary[idx][eventsStation]
                
                if(tripInstance[1] == BSN.C_ARRIVALS_IDX):
                    if(myBikeNetwork.getBikeStationObject(
                            StationId).pushBike() is False):
                        # This should never be the case as the
                        # relocation occurs within the scope of
                        # push/removeBike()
                        print(myBikeNetwork.getBikeStationObject(                            
                            StationId).availableBikes())
                        raise

                else:
                    if(myBikeNetwork.getBikeStationObject(
                            StationId).removeBike() is False):
                        # This should never be the case as the
                        # relocation occurs within the scope of
                        # push/removeBike()
                        print(myBikeNetwork.getBikeStationObject(
                            StationId).availableBikes())
                        raise
    

In [15]:
def getNetworkRiskStatus(bikeNetwork):
    # StatusBike Network -> [StationId, NumberofAvailableBikes]
    StatusBikeNetwork = [[int(s), int(bikeNetwork.getBikeStationObject(s).availableBikes())]
        for s in bikeNetwork.BikeStationsDict.keys() ]
    
    data = np.array(StatusBikeNetwork)
    
    # Identify Risk Stations by applying a formula np.max/min +- 1 whcih has calculated empirically
    # to provide a mechanism for Risk Assesment. It's performance is out of the scope of this
    # module for now.
    
    DockRiskStations = data[(data[:,1] >= (np.max(data[:,1]) - 1)) & ((data[:,1]) > C_OPTIMAL_LEVEL) ]
    BikeRiskStations = data[(data[:,1] <= (np.min(data[:,1]) + 1)) & ((data[:,1]) < C_OPTIMAL_LEVEL)]

    return DockRiskStations, BikeRiskStations

In [16]:
def upcomingTrips(time, dayItinerary):
    dayUpcoming = []
    for s in range(len(dayItinerary)):
        UpcomingTrips_bool = ((dayItinerary[s][:,0]*60) >= time) & ((dayItinerary[s][:,0]*60) < (time + (0.125*60)))
        
        # dayUpcoming -> [[list(Trips timestamps)], [dayItinerary bool indexers for location of those trips]]
        if(dayItinerary[s][UpcomingTrips_bool].size > 0):
            dayUpcoming.append([ dayItinerary[s][UpcomingTrips_bool],  UpcomingTrips_bool])
            
    return dayUpcoming

In [17]:
def getDistanceBetweenStations(RouteAnalysisSUMO, stationIdEdge1, stationIdEdge2):
    p1 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[stationIdEdge1]]
    p2 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[stationIdEdge2]]
    
    return GeometryClass.getDistance(p1,p2)

In [61]:
def retrieveRiskMeasurements(RiskStation, UpcomingInfo, myBikeNetwork, RouteAnalysisSUMO):
    NearStations = []
    for StationInfo in UpcomingInfo:
        for trip in StationInfo[0]:
            
            # Only departures will be analyzed by this function
            if((int(trip[1]) != BSN.C_ARRIVALS_IDX) & (int(trip[3]) in myBikeNetwork.getAllStationOnNetwork())):
                distance = \
                    getDistanceBetweenStations(
                        RouteAnalysisSUMO, 
                        myBikeNetwork.StationsOnNetwork.get(int(trip[3])), 
                        myBikeNetwork.StationsOnNetwork[RiskStation])
                
                # 400 meters were choosen as per Singla study where this is the
                # maximum distance users where willing to change the
                # destination station
                if((distance < 400.0) & (distance > 5.0)):
                    NearStations.append(str(int(trip[3])))
                    # print('One near station is: ' + str(int(trip[3])) + ' instead of: ' + str(RiskStation))
    return NearStations

In [77]:
C_MINUTES_IN_HOUR = 60
C_HOURS_IN_DAY = 24
C_OPTIMAL_LEVEL = 10 # 10 bikes / 20 docks

myBikeNetwork.resetNetwork()
for time in range(C_MINUTES_IN_HOUR*C_HOURS_IN_DAY):
    # Get Most Critical Stations
    DockRiskStations, BikeRiskStations = getNetworkRiskStatus(myBikeNetwork)
    # Get the itinerary for the next time delta
    UpcomingInfo = upcomingTrips(time, dayItinerary)
    
    if(time % (C_MINUTES_IN_HOUR) == 0):
        RiskMeasure = []
        for stationId in BikeRiskStations[:,0]:
            RiskMeasure_t =\
                retrieveRiskMeasurements(
                    stationId,
                    UpcomingInfo,
                    myBikeNetwork,
                    RouteAnalysisSUMO)
            if(RiskMeasure_t):
                # From all posible solutions, only the station with less bikes will be
                # selected as it is intended to avoid those stations to be empty
                RewardDestinationBikes =\
                    [myBikeNetwork.getBikeStationObject(s).availableBikes() for s in RiskMeasure_t]
                Idx = np.where(RewardDestinationBikes == np.min(RewardDestinationBikes))[0][0]
                print(str(RiskMeasure_t[Idx]) + ' replaces ' + str(stationId))
    
    # Update the Stations based on the dayItinerary provided / modified
    updateStatusOfStations(time,dayItinerary)
    
#     if(time % C_MINUTES_IN_HOUR == 0):
#         print('At time: ' + str(time) + ' minutes.') 
#         print('Dock Risk Stations: ')
#         print(DockRiskStations)
#         print('Bike Risk Stations: ')
#         print(BikeRiskStations)
#         print()


54 replaces 58
57 replaces 58
75 replaces 61
55 replaces 54
51 replaces 60
39 replaces 25
52 replaces 37
52 replaces 26
11 replaces 27
59 replaces 54
37 replaces 26
57 replaces 58
57 replaces 54
27 replaces 26
11 replaces 10
57 replaces 58
53 replaces 76
57 replaces 58
35 replaces 36
78 replaces 80
78 replaces 58
59 replaces 58
28 replaces 35


In [19]:
# DockRiskStations

In [26]:
# myBikeNetwork.StationsOnNetwork

In [36]:
DockRiskStations[:,0]

array([77, 31])

In [37]:
BikeRiskStations[:,0]

array([58, 54, 55, 35])

In [25]:
# UpcomingInfo

In [41]:
for i in BikeRiskStations[:,0]:
    print(retrieveRiskMeasurements(BikeRiskStations[:,0][0], UpcomingInfo, myBikeNetwork, RouteAnalysisSUMO))

        

[]
[]
[]
[]


In [ ]:
myBikeNetwork.StationsOnNetwork

In [ ]:
p1 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[myBikeNetwork.StationsOnNetwork[2]]]
p2 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[myBikeNetwork.StationsOnNetwork[10]]]


GeometryClass.getDistance(p1, p2)

In [ ]:
p1 = [3987.21,5576.04]
p2 = [3912.73,5573.69]

GeometryClass.getDistance(p1,p2)

In [ ]:
np.amax(np.array([-3, -2, -1, 0, 1, 2, 3]))

In [ ]:
np.amin(np.array([-3, -2, -1, 0, 1, 2, 3]))

In [ ]:
#Relocation events on every station
print([ (s, myBikeNetwork.getBikeStationObject(s).getStationStats()[0]) for s in myBikeNetwork.BikeStationsDict.keys() ])

In [ ]:
BikeFinalStatus = [s.availableBikes() for s in list(myBikeNetwork.BikeStationsDict.values())]
np.array(BikeFinalStatus) - np.array(BikeInitialStatus)

In [ ]:
C_BIKESTATION = 2

currentBikeStation =\
    myBikeNetwork.getBikeStationObject(C_BIKESTATION)
print(currentBikeStation)

In [ ]:
destinations = \
    list(
    currentBikeStation\
    .stationCDFDepartures\
    .wkDayDepartures[0]\
    .Destino_Id)
hist, bin_edges = \
    np.histogram(
        destinations, bins=np.max(destinations),
        density=True)

hist = np.append(hist, [0.0])
plt.figure(1)
plt.title('Station ID Destination of trips from station: ' + str(C_BIKESTATION))
plt.plot(bin_edges, hist)
plt.figure(2)
plt.title('CDF of Station ID Destination of trips from station: ' + str(C_BIKESTATION))
plt.plot(bin_edges, BSCDF.computeCDF(hist,bin_edges))



In [ ]:
C_NUMBER_OF_TRIPS = 50
C_WEEKDAY = 5

Arrivals = np.empty(0)
Departures = np.empty(0)
for i in range(C_NUMBER_OF_TRIPS):
    BikeStationTrips = \
        currentBikeStation.getTripsOnWeekday(C_WEEKDAY)

    tArrivals =\
        BikeStationTrips[
            BikeStationTrips[:, 1] ==
            BSN.C_ARRIVALS_IDX]
    Arrivals = np.r_[Arrivals, tArrivals[:,0]]


    tDepartures =\
        BikeStationTrips[
            BikeStationTrips[:, 1] == 
            BSN.C_DEPARTURES_IDX]
    Departures = np.r_[Departures, tDepartures[:,0]]


In [ ]:
BikeStationTrips

In [ ]:
plt.figure(1)
plt.title('Arrivals Station: ' + 
    currentBikeStation.getStationId())
plt.hist(Arrivals)
print(np.mean(Arrivals))

plt.figure(2)
plt.title(
    'Departures Station: ' + 
    currentBikeStation.getStationId())
plt.hist(Departures)
print(np.mean(Departures))

In [ ]:
ahist, abin_edges = \
    np.histogram(
        Arrivals, bins=24 * 8,
        density=True,
        range=(0.0, 24.0))
ahist = np.append(ahist, [0.0])

In [ ]:
dhist, dbin_edges = \
    np.histogram(
        Departures, bins=24 * 8,
        density=True,
        range=(0.0, 24.0))
dhist = np.append(dhist, [0.0])

In [ ]:
x =\
currentBikeStation.stationCDFArrivals\
.wkDayDepartures_cdf[C_WEEKDAY][0]

y =\
currentBikeStation.stationCDFArrivals\
.wkDayDepartures_cdf[C_WEEKDAY][1]

acdf = BSCDF.computeCDF(
            ahist,abin_edges)

plt.plot(x/3600,y)
plt.plot(24.0*np.arange(len(acdf))/len(acdf), acdf)

In [ ]:
x =\
currentBikeStation.stationCDFDepartures\
.wkDayDepartures_cdf[C_WEEKDAY][0]

y =\
currentBikeStation.stationCDFDepartures\
.wkDayDepartures_cdf[C_WEEKDAY][1]

dcdf = BSCDF.computeCDF(
            dhist,dbin_edges)

plt.plot(x/3600,y)
plt.plot(24.0*np.arange(len(dcdf))/len(dcdf), dcdf)